In [19]:
import json
from pathlib import Path
from scipy import sparse
import numpy as np
import pandas as pd

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('../raw_data/AI4Code')

In [20]:
paths = list((data_dir / 'train').glob('*.json'))

async def get_num_cells(path):
    return pd.read_json(path).size

num_cells = []
for path in paths:
    res = await get_num_cells(path)
    num_cells.append(res)

#num_cells = pd.Series([get_num_cells(nb_path) for nb_path in paths])
#num_cells

In [21]:
NUM_TRAIN = 20


def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=path.stem)
        .rename_axis('cell_id')
    )


paths_train = list((data_dir / 'train_data').glob('*.json'))[:NUM_TRAIN]
notebooks_train = [
    read_notebook(path) for path in paths_train
]
df = (
    pd.concat(notebooks_train)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)

df

cell_type                                                                                                                   source
id             cell_id                                                                                                                                    
0cf829b9d4e933 44d682f1      code  import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns\nimport time, os, warnings\ncolor = sns.c...
               29bddfce      code                                                                                                       inspections.head()
               5dc02636      code                                                                                                   inspections.describe()
               df0f0661      code                                                                                                       inspections.info()
               815fae86      code  fig, ax  = plt.subplots(2, 1, figsize = (10, 8))\nsns.boxplot(inspections["score"], ax = ax[0])\nax[0].set_title("Bo...
...                           ...                                                                                                                      ...
eaaea85f34a899 1c946fdb  markdown  ### Placa vertical\n\nA fin de obtener el coeficiente de transferencia de calor para la condensación en película sob...
               77134fab  markdown                                           *Alejandro Gronskis, julio 2020*\n\n*Juan D'Adamo, diciembre 2020*\n\n\n\n\n\n
               0d1da764  markdown  ### Placa horizontal cara hacia abajo\n\nA fin de obtener el coeficiente de transferencia de calor para la condensac...
               d59c0507  markdown  ## Efecto causado por cambios en la temperatura de pared\n\nA continuación, procuramos construir las funciones $\dot...
               49a9924a  markdown  # References\n\n<a name="cite-incropera2007fundamentals"/><sup>[^](#ref-1) </sup>Incropera, Frank P and Lavine, Adri...

[960 rows x 2 columns]

In [22]:
# Get an example notebook
nb_id = df.index.unique('id')[6]
print('Notebook:', nb_id)

print("The disordered notebook:")
nb = df.loc[nb_id, :]
display(nb)
print()

Notebook: 6763cde90c858f
The disordered notebook:


,cell_type,source
cell_id,,
e613ddcc,code,# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...
c6fc5d53,code,import pandas as pd\nimport numpy as np\n\n# for data visualizations\n\nimport matplotlib.pyplot as plt\nimport seab...
6ec8c949,code,"data = pd.read_csv('../input/graduate-admissions/Admission_Predict.csv', index_col = 'Serial No.')\ndata2 = pd.read_..."
18fcfa33,code,"data = pd.concat([data, data2], axis = 0)"
001c7706,code,data.head()
...,...,...
7417c649,markdown,Feature Scaling/Standardization
9868eaa1,markdown,## This notebook is about the implementation of the Model Stacking Method which is widely used in Data Analysis.\n\n...
64172015,markdown,## Model Deployed on Heroku @ https://graduate-admission-chances.herokuapp.com/


In [23]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
)  # Split the string representation of cell_ids into a list

df_orders

,cell_order
id,
00001756c60be8,1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 77e56113 2eefe0ef 1ae087ab 0beab1cd 8ffe0b25 9a78ab76 0d136e08 8a4c95d1...
00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 51e3cd89 2600b4eb 75b65993 cf195f8b 25699d02 72b3201a f2c750d3 de148b56...
0001bdd4021779,3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 073e27e5 015d52a4 ad7679ef 7fde4f04 07c52510 0a1a7a39 0bcd3fef 58bf360b
0001daf4c2c76d,97266564 a898e555 86605076 76cc2642 ef279279 df6c939f 2476da96 00f87d0a ae93e8e6 58aadb1d d20b0094 986fd4f1 b4ff1015...
0002115f48f982,9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a3188e54 b3f6e12d ee7655ca 84125b7a
...,...
fffc30d5a0bc46,09727c0c ff1ea6a0 ddfef603 a01ce9b3 3ba953ee bf92a015 f4a0492a 095812e6 53125cfe aa32a700 63340e73 06d8c952 400ceb37...
fffc3b44869198,978a5137 faa48f03 28dfb12a eea2e812 64fef97c 4e0d1510 58e68f2c 8784e700 4bd5a4cf dc14bfec 2aff7603 8047ded2 ac536a5b...
fffc63ff750064,5015c300 411b85d9 8238198c f4781d1d b5532930 e1f223e5 e7e67119 4aaf741d 7229cce6 a7fa3628 e4c2fa86 1f8f9d14 7a52357a...


In [24]:
df_orders = df_orders['cell_order'].str.split()

In [25]:
# Get the correct order
cell_order = df_orders.loc[nb_id]

print("The ordered notebook:")
nb.loc[cell_order, :]

The ordered notebook:


,cell_type,source
cell_id,,
6523f8e1,markdown,## Model Stacking
9868eaa1,markdown,## This notebook is about the implementation of the Model Stacking Method which is widely used in Data Analysis.\n\n...
64172015,markdown,## Model Deployed on Heroku @ https://graduate-admission-chances.herokuapp.com/
e613ddcc,code,# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...
5d658593,markdown,Necessary Library Imports
...,...,...
fbb01ac1,code,dir(meta_model)
7e6d24fb,code,# saving the ml model in pickle format
d01ddc52,code,files = [file for file in os.listdir('/kaggle/working') if file.endswith('.pkl')]


In [36]:
cell_order

['62194bb3',
 '2a744929',
 '4669d3a9',
 '64ee9541',
 '7bb9b2d1',
 '034aa67c',
 '6e428fcf',
 '1b9e81a3',
 '0d22a837',
 'f0a52dc8',
 '0d1da764',
 '033cbd56',
 'c5ef3ca5',
 'e031694b',
 '1c946fdb',
 '0648be54',
 '82f877f4',
 'd59c0507',
 '8d5b9cbd',
 '930358ce',
 '358007e0',
 '4fac3e69',
 '06c99d3d',
 '2ddeb9af',
 '77134fab',
 '68bf700d',
 '49a9924a']

In [26]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

cell_ranks = get_ranks(cell_order, list(nb.index))
nb.insert(0, 'rank', cell_ranks)

nb

,rank,cell_type,source
cell_id,,,
e613ddcc,3,code,# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...
c6fc5d53,5,code,import pandas as pd\nimport numpy as np\n\n# for data visualizations\n\nimport matplotlib.pyplot as plt\nimport seab...
6ec8c949,7,code,"data = pd.read_csv('../input/graduate-admissions/Admission_Predict.csv', index_col = 'Serial No.')\ndata2 = pd.read_..."
18fcfa33,8,code,"data = pd.concat([data, data2], axis = 0)"
001c7706,9,code,data.head()
...,...,...,...
7417c649,18,markdown,Feature Scaling/Standardization
9868eaa1,1,markdown,## This notebook is about the implementation of the Model Stacking Method which is widely used in Data Analysis.\n\n...
64172015,2,markdown,## Model Deployed on Heroku @ https://graduate-admission-chances.herokuapp.com/


In [27]:
df_orders_ = df_orders.to_frame().join(
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
    how='right',
)

ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}

df_ranks = (
    pd.DataFrame
    .from_dict(ranks, orient='index')
    .rename_axis('id')
    .apply(pd.Series.explode)
    .set_index('cell_id', append=True)
)

df_ranks

rank
id             cell_id      
0cf829b9d4e933 44d682f1    0
               29bddfce    3
               5dc02636    4
               df0f0661    5
               815fae86    9
...                      ...
eaaea85f34a899 1c946fdb   14
               77134fab   24
               0d1da764   10
               d59c0507   17
               49a9924a   26

[960 rows x 1 columns]

In [28]:
df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')
df_ancestors

,ancestor_id,parent_id
id,,
00001756c60be8,945aea18,NaN
00015c83e2717b,aa2da37e,317b65d12af9df
0001bdd4021779,a7711fde,NaN
0001daf4c2c76d,090152ca,NaN
0002115f48f982,272b483a,NaN
...,...,...
fffc30d5a0bc46,6aed207b,NaN
fffc3b44869198,a6aaa8d7,NaN
fffc63ff750064,0a1b5b65,NaN


In [29]:
from sklearn.model_selection import GroupShuffleSplit
NVALID = 0.1  # size of validation set

splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)

# Split, keeping notebooks with a common origin (ancestor_id) together
ids = df.index.unique('id')
ancestors = df_ancestors.loc[ids, 'ancestor_id']
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors))
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

In [30]:
df_train

cell_type                                                                                                                   source
id             cell_id                                                                                                                                    
0cf829b9d4e933 44d682f1      code  import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns\nimport time, os, warnings\ncolor = sns.c...
               29bddfce      code                                                                                                       inspections.head()
               5dc02636      code                                                                                                   inspections.describe()
               df0f0661      code                                                                                                       inspections.info()
               815fae86      code  fig, ax  = plt.subplots(2, 1, figsize = (10, 8))\nsns.boxplot(inspections["score"], ax = ax[0])\nax[0].set_title("Bo...
...                           ...                                                                                                                      ...
eaaea85f34a899 1c946fdb  markdown  ### Placa vertical\n\nA fin de obtener el coeficiente de transferencia de calor para la condensación en película sob...
               77134fab  markdown                                           *Alejandro Gronskis, julio 2020*\n\n*Juan D'Adamo, diciembre 2020*\n\n\n\n\n\n
               0d1da764  markdown  ### Placa horizontal cara hacia abajo\n\nA fin de obtener el coeficiente de transferencia de calor para la condensac...
               d59c0507  markdown  ## Efecto causado por cambios en la temperatura de pared\n\nA continuación, procuramos construir las funciones $\dot...
               49a9924a  markdown  # References\n\n<a name="cite-incropera2007fundamentals"/><sup>[^](#ref-1) </sup>Incropera, Frank P and Lavine, Adri...

[826 rows x 2 columns]

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Training set
tfidf = TfidfVectorizer(min_df=0.01)
X_train = tfidf.fit_transform(df_train['source'].astype(str))
# Rank of each cell within the notebook
y_train = df_ranks.loc[ids_train].to_numpy()
# Number of cells in each notebook
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()

In [32]:
# Add code cell ordering
X_train = sparse.hstack((
    X_train,
    np.where(
        df_train['cell_type'] == 'code',
        df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))
print(X_train.shape)

(826, 371)


/var/folders/04/sd763rcx6rd1bxvl33_fm1s80000gn/T/ipykernel_53339/2052814572.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,


In [33]:
from xgboost import XGBRanker

model = XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)

model.fit(X_train, y_train, group=groups)

XGBoostError: [17:11:06] /Users/runner/work/xgboost/xgboost/src/common/ranking_utils.h:355: Check failed: label_is_valid: Relevance degress must be lesser than or equal to 31 when the exponential NDCG gain function is used. Set `ndcg_exp_gain` to false to use custom DCG gain.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000002a4cf0994 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000002a4d7bc68 xgboost::ltr::NDCGCache::InitOnCPU(xgboost::Context const*, xgboost::MetaInfo const&) + 1420
  [bt] (2) 3   libxgboost.dylib                    0x00000002a4e85550 xgboost::ltr::NDCGCache::NDCGCache(xgboost::Context const*, xgboost::MetaInfo const&, xgboost::ltr::LambdaRankParam const&) + 168
  [bt] (3) 4   libxgboost.dylib                    0x00000002a4ea1580 xgboost::obj::LambdaRankObj<xgboost::obj::LambdaRankNDCG, xgboost::ltr::NDCGCache>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo cons